In [1]:
import pandas as pd
from decimal import Decimal
import json
import math
from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-5

In this demo we investigate the **Fast Router performance** against test orders

### Current Issues
- tradeBySource orderIndex=1 example 53

Initialize a fast simulator

In [2]:
filename = '20221219_tradeByTargetAmount.json'
if 'Target' in filename:
    is_by_target = True
else:
    is_by_target = False
f = open(filename , "r")
bm = f.read()
bm_json = json.loads(bm)
print(len(bm_json))

120


In [3]:
def decimalize_dict(dicta):
    return({k:Decimal(v) for k,v in dicta.items()})

def calc_y_int(liquidity, highestRate, lowestRate, marginalRate):
    return(liquidity * (highestRate.sqrt() - lowestRate.sqrt()) / (marginalRate.sqrt() - lowestRate.sqrt()))

def parse_order(order0, order_count):
    order0 = decimalize_dict(order0)
    order0['delta_liquidity'] = order0['newLiquidity'] - order0['liquidity']
    order0['y_int'] = calc_y_int(order0['liquidity'], order0['highestRate'], order0['lowestRate'], order0['marginalRate'])
    order0['fix_trade'] = order0['y_int'] - order0['liquidity']
    order0['y_int_minus_fixtrade'] = order0['y_int'] - order0['fix_trade']
    order0['order'] = order_count
    order0_df = pd.DataFrame.from_dict(order0, 'index', columns=[f'{order_count}'])
    return(order0_df)

def parse_strategy(strategy, strat_count, order_count):
    orders = pd.DataFrame()
    for order in strategy:
        order_df = parse_order(order, order_count)
        orders = pd.concat([orders,order_df.T])
        order_count += 1
    # orders['pair'] = [Sim.carbon_pair.pair_slash, Sim.carbon_pair.reverse.pair_slash]
    orders['strategy'] = strat_count
    strat_count += 1
    return(orders, strat_count, order_count)


In [4]:
len(bm_json)

120

In [5]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

setup_index = 7

strats = pd.DataFrame()
strat_count = 0
order_count = 0
for strategy in bm_json[setup_index]['strategies']:
    orders, strat_count, order_count = parse_strategy(strategy, strat_count, order_count)
    strats = pd.concat([strats, orders])
strats


,liquidity,lowestRate,highestRate,marginalRate,newLiquidity,newMarginalRate,delta_liquidity,y_int,fix_trade,y_int_minus_fixtrade,order,strategy
0,867692,8.000000000000,9.000000000000,9.000000000000,868192,9.000000000000,500,867692.000000000000000000000000000000000000000...,2E-122,867692.000000000000000000000000000000000000000...,0,0
1,2955962,1.000000000000,2.000000000000,1.997233115801,2954962,1.996838002976,-1000,2962962.00000052006112479354298586116775999950...,7000.00000052006112479354298586116775999950584...,2955962.00000000000000000000000000000000000000...,1,0
2,991647,9.000000000000,10.000000000000,10.000000000000,992314,10.000000000000,667,991647.000000000000000000000000000000000000000...,3E-122,991647.000000000000000000000000000000000000000...,2,1
3,3938616,2.000000000000,3.000000000000,2.996656581157,3936616,2.996099525917,-2000,3950616.00000145969866196997725233571309317656...,12000.0000014596986619699772523357130931765626...,3938616.00000000000000000000000000000000000000...,3,1
4,1114854,10.000000000000,11.000000000000,11.000000000000,1115604,11.000000000000,750,1114854.00000000000000000000000000000000000000...,0E-121,1114854.00000000000000000000000000000000000000...,4,2
5,4923270,3.000000000000,4.000000000000,3.996745078680,4920270,3.996094253399,-3000,4938270.00000227764134656348376076751710117889...,15000.0000022776413465634837607675171011788984...,4923270.00000000000000000000000000000000000000...,5,2
6,1237760,11.000000000000,12.000000000000,12.000000000000,1238560,12.000000000000,800,1237760.00000000000000000000000000000000000000...,0E-121,1237760.00000000000000000000000000000000000000...,6,3
7,5909924,4.000000000000,5.000000000000,4.997149939489,5905924,4.996437551316,-4000,5925923.99999795185015513426267725419276402297...,15999.9999979518501551342626772541927640229774...,5909924.00000000000000000000000000000000000000...,7,3
8,1360515,12.000000000000,13.000000000000,13.000000000000,1361348,13.000000000000,833,1360515.00000000000000000000000000000000000000...,0E-121,1360515.00000000000000000000000000000000000000...,8,4
9,6898578,5.000000000000,6.000000000000,5.997731746083,6893578,5.996975756740,-5000,6913578.00000198361151673797008311063461033130...,15000.0000019836115167379700831106346103313024...,6898578.00000000000000000000000000000000000000...,9,4


In [6]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

for i in strats.order[::2]:
    FastSim.add_strategy('USDC', strats.y_int[i], strats.highestRate[i], strats.lowestRate[i], strats.y_int[i+1], 1/strats.highestRate[i+1], 1/strats.lowestRate[i+1])
    if strats.fix_trade[i] > 0.0000001:
        print(i)
        FastSim.amm_sells('USDC', amt=strats.fix_trade[i], execute=True, use_positions=[i], use_positions_matchlevel=[i])['trades']

for i in strats.order[1::2]:
    if strats.fix_trade[i] > 0.0000001:
        print(i)
        FastSim.amm_sells('ETH', amt=strats.fix_trade[i], execute=True, use_positions=[i], use_positions_matchlevel=[i])['trades']

initial_orders = FastSim.state()['orders']
initial_orders

1
3
5
7
9
11
13


,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,871194.42354,871194.42354,USDC,False,9.00000,8.00000,9.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,2962962.00000,2955962.00000,ETH,False,0.50000,1.00000,0.50069,USDC per ETH,0
2,2,ETHUSDC,USDC,995649.23081,995649.23081,USDC,False,10.00000,9.00000,10.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,3950616.00000,3938616.00000,ETH,False,0.33333,0.50000,0.33371,USDC per ETH,2
4,4,ETHUSDC,USDC,1118605.52668,1118605.52668,USDC,False,11.00000,10.00000,11.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,4938270.00000,4923270.00000,ETH,False,0.25000,0.33333,0.25020,USDC per ETH,4
6,6,ETHUSDC,USDC,1240960.91241,1240960.91241,USDC,False,12.00000,11.00000,12.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,5925924.00000,5909924.00000,ETH,False,0.20000,0.25000,0.20011,USDC per ETH,6
8,8,ETHUSDC,USDC,1363015.47269,1363015.47269,USDC,False,13.00000,12.00000,13.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,6913578.00000,6898578.00000,ETH,False,0.16667,0.20000,0.16673,USDC per ETH,8


In [7]:
FastSim.state()['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #1,False,True,None,7000.00000,ETH,3502.42354,USDC,ETHUSDC,1,1,0.50035,USDC per ETH
0,0,0,A,AMM sells 7000ETH buys 3502USDC,True,True,None,7000.00000,ETH,3502.42354,USDC,ETHUSDC,[1],1,0.50035,USDC per ETH
0,1.0,1,0,route #3,False,True,None,12000.00000,ETH,4002.23081,USDC,ETHUSDC,3,1,0.33352,USDC per ETH
0,1,1,A,AMM sells 12000ETH buys 4002USDC,True,True,None,12000.00000,ETH,4002.23081,USDC,ETHUSDC,[3],1,0.33352,USDC per ETH
0,2.0,2,0,route #5,False,True,None,15000.00000,ETH,3751.52668,USDC,ETHUSDC,5,1,0.25010,USDC per ETH
0,2,2,A,AMM sells 15000ETH buys 3752USDC,True,True,None,15000.00000,ETH,3751.52668,USDC,ETHUSDC,[5],1,0.25010,USDC per ETH
0,3.0,3,0,route #7,False,True,None,16000.00000,ETH,3200.91241,USDC,ETHUSDC,7,1,0.20006,USDC per ETH
0,3,3,A,AMM sells 16000ETH buys 3201USDC,True,True,None,16000.00000,ETH,3200.91241,USDC,ETHUSDC,[7],1,0.20006,USDC per ETH
0,4.0,4,0,route #9,False,True,None,15000.00000,ETH,2500.47269,USDC,ETHUSDC,9,1,0.16670,USDC per ETH
0,4,4,A,AMM sells 15000ETH buys 2500USDC,True,True,None,15000.00000,ETH,2500.47269,USDC,ETHUSDC,[9],1,0.16670,USDC per ETH


In [8]:
bm_json[setup_index]['actions']

[{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'},
 {'strategyId': 1, 'orderIndex': 0, 'tokenAmount': '2000'},
 {'strategyId': 2, 'orderIndex': 0, 'tokenAmount': '3000'},
 {'strategyId': 3, 'orderIndex': 0, 'tokenAmount': '4000'},
 {'strategyId': 4, 'orderIndex': 0, 'tokenAmount': '5000'},
 {'strategyId': 5, 'orderIndex': 0, 'tokenAmount': '6000'},
 {'strategyId': 6, 'orderIndex': 0, 'tokenAmount': '7000'},
 {'strategyId': 7, 'orderIndex': 0, 'tokenAmount': '8000'}]

In [9]:
if len(bm_json[setup_index]['actions']) == 1:
    actionList = [bm_json[setup_index]['actions']]
else:
    actionList = bm_json[setup_index]['actions']
    
print(f'is_by_target {is_by_target}')
for action in actionList:
    print(action)
    targetOrder = strats.query(f"strategy=={action['strategyId']}")
    targetOrderId = targetOrder['order'].values[0]
    targetOrderToken = FastSim.state()['orders'].query(f"id=={targetOrderId}")['tkn'].values[0]
    tokenAmount = Decimal(action['tokenAmount'])
    if not is_by_target:
        if action['orderIndex'] == 1:
            FastSim.amm_buys(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        else:    
            targetOrderId += 1
            FastSim.amm_buys(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_source
        print(targetOrderToken, targetOrderId, tokenAmount)
    else:
        if action['orderIndex'] == 1:
            FastSim.amm_sells(targetOrderToken,tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        else:
            targetOrderId += 1
            FastSim.amm_sells(FastSim.carbon_pair.other(targetOrderToken),tokenAmount, execute=True, use_positions=[targetOrderId], use_positions_matchlevel=[targetOrderId])['trades']  # route_trade_by_target
        print(FastSim.carbon_pair.other(targetOrderToken), targetOrderId, tokenAmount)
    

is_by_target True
{'strategyId': 0, 'orderIndex': 0, 'tokenAmount': '1000'}
ETH 1 1000
{'strategyId': 1, 'orderIndex': 0, 'tokenAmount': '2000'}
ETH 3 2000
{'strategyId': 2, 'orderIndex': 0, 'tokenAmount': '3000'}
ETH 5 3000
{'strategyId': 3, 'orderIndex': 0, 'tokenAmount': '4000'}
ETH 7 4000
{'strategyId': 4, 'orderIndex': 0, 'tokenAmount': '5000'}
ETH 9 5000
{'strategyId': 5, 'orderIndex': 0, 'tokenAmount': '6000'}
ETH 11 6000
{'strategyId': 6, 'orderIndex': 0, 'tokenAmount': '7000'}
ETH 13 7000
{'strategyId': 7, 'orderIndex': 0, 'tokenAmount': '8000'}
ETH 15 8000


In [10]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

final_orders = FastSim.state()['orders']
final_orders

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,871695.16575,871695.16575,USDC,False,9.00000,8.00000,9.00000,USDC per ETH,1
1,1,ETHUSDC,ETH,2962962.00000,2954962.00000,ETH,False,0.50000,1.00000,0.50079,USDC per ETH,0
2,2,ETHUSDC,USDC,996316.70333,996316.70333,USDC,False,10.00000,9.00000,10.00000,USDC per ETH,3
3,3,ETHUSDC,ETH,3950616.00000,3936616.00000,ETH,False,0.33333,0.50000,0.33377,USDC per ETH,2
4,4,ETHUSDC,USDC,1119356.19859,1119356.19859,USDC,False,11.00000,10.00000,11.00000,USDC per ETH,5
5,5,ETHUSDC,ETH,4938270.00000,4920270.00000,ETH,False,0.25000,0.33333,0.25024,USDC per ETH,4
6,6,ETHUSDC,USDC,1241761.42574,1241761.42574,USDC,False,12.00000,11.00000,12.00000,USDC per ETH,7
7,7,ETHUSDC,ETH,5925924.00000,5905924.00000,ETH,False,0.20000,0.25000,0.20014,USDC per ETH,6
8,8,ETHUSDC,USDC,1363849.17372,1363849.17372,USDC,False,13.00000,12.00000,13.00000,USDC per ETH,9
9,9,ETHUSDC,ETH,6913578.00000,6893578.00000,ETH,False,0.16667,0.20000,0.16675,USDC per ETH,8


In [11]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
compare_results = pd.concat([strats[['newLiquidity', 'newMarginalRate']].reset_index(drop=True), final_orders[['tkn', 'y', 'p_marg']].reset_index(drop=True)], axis=1)
compare_results.columns = ['json_liquidity', 'json_price', 'tkn', 'sim_liquidity', 'sim_price']
compare_results = pd.concat([compare_results.reset_index(drop=True), strats[['delta_liquidity']].reset_index(drop=True), (final_orders['y'] - initial_orders['y']).reset_index(drop=True)], axis=1)

compare_results = compare_results[['tkn','json_price', 'sim_price', 'json_liquidity', 'sim_liquidity', 'delta_liquidity', 'y']]
compare_results.loc[:,'sim_price'] = [1/compare_results.sim_price[i] if i%2==1 else compare_results.sim_price[i] for i in compare_results.index]
# compare_results = compare_results.astype(float)
compare_results

,tkn,json_price,sim_price,json_liquidity,sim_liquidity,delta_liquidity,y
0,USDC,9.000000000000,9.00000,868192,871695.16575,500,500.74221
1,ETH,1.996838002976,1.99684,2954962,2954962.00000,-1000,-1000.00000
2,USDC,10.000000000000,10.00000,992314,996316.70333,667,667.47252
3,ETH,2.996099525917,2.99610,3936616,3936616.00000,-2000,-2000.00000
4,USDC,11.000000000000,11.00000,1115604,1119356.19859,750,750.67192
5,ETH,3.996094253399,3.99609,4920270,4920270.00000,-3000,-3000.00000
6,USDC,12.000000000000,12.00000,1238560,1241761.42574,800,800.51333
7,ETH,4.996437551316,4.99644,5905924,5905924.00000,-4000,-4000.00000
8,USDC,13.000000000000,13.00000,1361348,1363849.17372,833,833.70103
9,ETH,5.996975756740,5.99698,6893578,6893578.00000,-5000,-5000.00000
